# COLLECTING DATA
---

Nhiệm vụ của phần này là thực hiện lấy dữ liệu từ thông tin imdbid và title trong file ids_titles.txt.

## 1. IMPORT THƯ VIỆN CẦN THIẾT

In [32]:
import pandas as pd
import time
import os
import requests
from bs4 import BeautifulSoup
import re

## 2. HÀM LẤY DỮ LIỆU CỦA TRANG `OMDB API`

Phần dưới này để đổi thông tin người thực hiện

In [35]:
# chia 3 ra nha mina hong no k cho minh lay :D
# từ từ để ngâm cú chứ nhìn này trông cx hơi ngu :v
member_part = {
    "Trinh": {
        "part": 1,
        "api_key": None, # đk api xong áp vô nè
        "start": 0,
        "end": 628
    },

    "Thi": {
        "part": 2,
        "api_key": None, # đk api xong áp vô nè
        "start": 808,
        "end": 1258
    },

    "MinhUyen": {
        "part": 3,
        "api_key": 'd4c83dec',
        "start": 1259,
        "end": 1887
    } 
}

In [36]:
# Điền tên mn vào đây theo từ điển nè
info = member_part["MinhUyen"]

part = info["part"]
apiKey = info["api_key"]
start = info["start"]
end = info["end"]

Hàm này trả về link từ id hoặc tiêu đề phim

In [58]:
def convert_url(item: str):
    if item.startswith("tt"):
        url = f'https://www.omdbapi.com/?i={item}&apikey={key}'
    else: 
        title = item.split(' __ ')[0]
        match = re.search(r':(.*)', title)
        if match:
            title = match.group(1)
            title = re.sub(r'[^a-zA-Z0-9\s]', '', title)
            
        year = item.split(' __ ')[1]
        title = '+'.join(title.split(' '))
        url = f'https://www.omdbapi.com/?t={title}&y={year}&apikey={apiKey}'

    return url

Hàm này thì thu thập dữ liệu

In [ ]:
def collectMovieData(imdbIds_titles, key = 0, start = 0, end = 0):
    title = []
    rated = []
    released = []
    runtime = []
    genre = []
    metaScore = []
    rating = []
    vote = []
    boxCffice = []

    for item in imdbIds_titles[start:end]:
        print(f"[LOG]: Collecting data of {item}")
        url = convert_url(item)
        trying = 1

        while True:    
            respond = requests.get(url)
            if respond.status_code == 200:
                data = respond.json()
                if data.get("Response") == "False": # bỏ qua những phim không thể tim thấy trong trang
                    break

                title.append(data['Title'])
                rated.append(data['Rated'])
                released.append(data['Released'])
                runtime.append(data['Runtime'])
                genre.append(data['Genre'])
                metaScore.append(data['Metascore'])
                rating.append(data['imdbRating'])
                vote.append(data['imdbVotes'])
                boxCffice.append(data['BoxOffice'])
                break
                
            else:
                trying += 1
                if trying > 3:
                    print(f"Request fail after 3 retries with code {respond.status_code}")

                time.sleep(1)

    movieDataframe = pd.DataFrame({
            "Title": title,
            "Rated": rated,
            "Released": released,
            "Runtime": runtime,
            "Genre": genre,
            "Metascore": metaScore,
            "IMDb Rating": rating,
            "IMDb Votes": vote,
            "Domestic Revenue": boxCffice

        })

    return movieDataframe

Phần này lấy các title và id

In [ ]:
ids_titles = []

with open('../Data/ids_title.txt', 'r') as f:
    ids_titles = [line.strip() for line in f]

này là thu thập dữ liệu nè

In [ ]:
df1 = collectMovieData(ids_titles, key=apiKey)
df1

## 3. LƯU DỮ LIỆU THÔ VÀO FILE CSV

In [ ]:
file_name = os.path.join("./Data/", f"raw_movie_data_{str(part)}.csv")
df1.to_csv(file_name)